### **Exercício 2**

Desenvolva uma GIC para definir uma linguagem que permita escrever listas mistas de números e palavras, de tal forma que as 3 frases abaixo sejam frases válidas dessa linguagem:


Lista 1, 2, agora, 3, 4, fim, 7, 8.

Resolva as seguintes alíneas recorrendo ao uso de Transformer :

(a) Calcule, retorne e imprima quantos elementos existem numa lista.

(b) Identifique, retorne e imprima o elemento que mais se repete numa lista.

(c) Calcule, retorne e imprima a soma de todos os números que se encontrem entre as palavras agora e fim de uma lista.

(d) Verifique se uma lista é válida de acordo com os seguintes parâmetros :

    Caso a palavra "agora" ocorra então a palavra "fim" também tem de aparecer posteriormente na lista e o contrário também.
    Entre "agora" e "fim" tem de aparecer pelo menos um número.

In [8]:
from lark import Lark, Transformer
from collections import Counter



grammar = '''


// Regras Sintáticas
start: LISTA (conteudo)+ PF

conteudo: elemento (VIR elemento)+

elemento: PALAVRA 
        | NUM 
        

// Regras Lexicográficas
NUM:  /[0-9]+/
LISTA: /(?i:lista)/
PALAVRA: /[a-zA-Z]+/ 
PF: "." 
VIR: "," 

// Tratamento dos espaços em branco
%import common.WS
%ignore WS
%import common.NUMBER
%import common.WS_INLINE
%ignore WS_INLINE
'''




class ValidacaoTransformer(Transformer):

    def __init__(self):
        self.contador_numeros = 0
        self.contador_flags = 0
        self.erro = False  

    def start(self, elementos):
        if self.contador_flags > 0: 
            raise ValueError("Erro Semântico: Falta da condição 'fim'.")
        if self.contador_flags < 0: 
            raise ValueError("Erro Semântico: Falta da condição 'agora'.")
        

        return elementos  

    def elemento(self, elemento): 
        return elemento

    def NUM(self, numero):
        #print("NUM:", numero)
        self.contador_numeros +=1 
        return int(numero)
    
    def PALAVRA(self, palavra):
        #print("PALAVRA:", palavra)

        palavra = palavra.lower()  
        
        if palavra == "fim":
            if self.contador_flags == 0:
                raise ValueError("Erro Semântico: 'fim' encontrado sem um 'agora' correspondente.")
            if self.contador_numeros == 0: 
                raise ValueError("Erro Semântico: Entre 'agora' e 'fim' tem de aparecer pelo menos um número.")
            self.contador_flags -= 1 
            self.contador_numeros = 0 

        elif palavra == "agora":
            self.contador_flags += 1 
        
        return palavra
    
    def VIR (self, virgula): 
        virgula = virgula.lower()
        return virgula 




parser = Lark(grammar, parser='lalr', transformer=None)


# Função de parsing
def parse_input(input_text):
    try:
        tree = parser.parse(input_text)
        
        transformer = ValidacaoTransformer()
        result = transformer.transform(tree)
    
        if transformer.erro: 
            print("\nParsing falhou: Uma ou mais restrições foram violadas.")
        else:
            print("\nParsing bem-sucedido: Todas as restrições foram atendidas.")

    except Exception as e: 
        print (e)

if __name__ == "__main__":
    test_cases = [
    "Lista 1, 2, agora, 3, 4, fim, 7, 8.",
    "Lista 1, 2, agora, 3, 4, fim, 7, 8, 7.", 
    "Lista  1 agora 2 agora 3 fim 4 fim 5 . ",
    "Lista  1, agora, 2, agora, 3, fim, 4, fim, 5 . ",
    "Lista  1, agora, 2, agora, 3, fim, 4, fim, 5, 6, agora, 10, fim . ",
    "LISTA 3 , agora, 3, 4   .",
    "LISTA agora, fim . ", 
    "LISTA agora,2,  fim . ", 
    "LISTA 1, 2, fim . ",
    "LisTa fim, fim, agora,  2, fim  . ", 


    ]



#    for input_text in test_cases:
#        print("\n\n\nInput:", input_text)
#        parse_input(input_text)


In [9]:

# ? start: LISTA (conteudo)+ PF
# ? 
# ? conteudo: elemento (VIR elemento)+
# ? 
# ? elemento: PALAVRA 
# ?         | NUM 
        

class calculo_Transformer(Transformer):

    def __init__(self):
      self.sinal=1
      self.erro = False  
      self.lista_elems = []
      self.soma_lista = 0 
      self.somador_on = False
      self.somador_number = 0 

    def start ( self,token ): 
        print(f"Numero de Elementos {len(self.lista_elems)}")
        print(f"Mais Comum {Counter(self.lista_elems).most_common()[0]}")
        print(f"Soma de Todos os Numeros entre agora e fim: {self.soma_lista}")
        pass

    def conteudo ( self,token ): 
        return token 
        
    def NUM (self, token):
        self.lista_elems.extend(token)

        if self.somador_on:
            self.soma_lista += int(token)
        return int(token.value)
    
    def LISTA (self, token):
        return token.value 

    def PALAVRA(self, palavra):
       

        palavra = palavra.lower()  
        
        if palavra == "fim":
            self.somador_number -= 1 
            if self.somador_number == 0: 
                self.somador_on = False

        elif palavra == "agora":
            self.somador_number += 1 
            if self.somador_number >=1: 
                self.somador_on = True
        


        return palavra
    

    def VIR (self, token):
        return token.value 
    def PF (self, token):
        return token.value 





parser = Lark(grammar, parser='lalr', transformer=None)
# Função de parsing
def parse_input(input_text):
    try:
        tree = parser.parse(input_text)
        
        transformer = ValidacaoTransformer()

        
        result = transformer.transform(tree)
    
        if transformer.erro: 
            print("\nParsing falhou: Uma ou mais restrições foram violadas.")
        else:
            transformer_caculo = calculo_Transformer()
            result_calculo= transformer_caculo.transform(tree)

    except Exception as e: 
        print (e)

if __name__ == "__main__":
    test_cases = [
    "Lista 1, 2, agora, 3, 4, fim, 7, 8.",
    "Lista 1, 2, agora, 3, 4, fim, 7, 8, 7.", 
    "Lista  1 agora 2 agora 3 fim 4 fim 5 . ",
    "Lista  1, agora, 2, agora, 3, fim, 4, fim, 5 . ",
    "Lista  1, agora, 2, agora, 3, fim, 4, fim, 5, 6, agora, 10, fim . ",
    "LISTA 3 , agora, 3, 4   .",
    "LISTA agora, fim . ", 
    "LISTA agora,2,  fim . ", 
    "LISTA 1, 2, fim . ",
    "LisTa fim, fim, agora,  2, fim  . ", 


    ]



    for input_text in test_cases:
        print("\n\n\nInput:", input_text)
        parse_input(input_text)




Input: Lista 1, 2, agora, 3, 4, fim, 7, 8.
Numero de Elementos 6
Mais Comum ('1', 1)
Soma de Todos os Numeros entre agora e fim: 7



Input: Lista 1, 2, agora, 3, 4, fim, 7, 8, 7.
Numero de Elementos 7
Mais Comum ('7', 2)
Soma de Todos os Numeros entre agora e fim: 7



Input: Lista  1 agora 2 agora 3 fim 4 fim 5 . 
Unexpected token Token('PALAVRA', 'agora') at line 1, column 10.
Expected one of: 
	* VIR




Input: Lista  1, agora, 2, agora, 3, fim, 4, fim, 5 . 
Numero de Elementos 5
Mais Comum ('1', 1)
Soma de Todos os Numeros entre agora e fim: 9



Input: Lista  1, agora, 2, agora, 3, fim, 4, fim, 5, 6, agora, 10, fim . 
Numero de Elementos 8
Mais Comum ('1', 2)
Soma de Todos os Numeros entre agora e fim: 19



Input: LISTA 3 , agora, 3, 4   .
Error trying to process rule "start":

Erro Semântico: Falta da condição 'fim'.



Input: LISTA agora, fim . 
Error trying to process rule "PALAVRA":

Erro Semântico: Entre 'agora' e 'fim' tem de aparecer pelo menos um número.



Input: LIS